In [17]:
import pandas as pd
import numpy as np
#from google.colab import drive
from statsmodels.tools.eval_measures import rmse

In [6]:
#drive.mount('/content/drive')

In [7]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [8]:
filtro_women = pd.read_excel('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Women_train_v3.xlsx?raw=true')

In [9]:
filtro_women.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter
0,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,24-10-2018,14,2018-10-24,24,10,2018,ador top hint lace key hole back pale pink 1x ...,ava-viv blous,target
1,24K GOLD plated rose,1,Women/Jewelry/Necklaces,Other,44.0,0,Complete with certificate of authenticity,16-4-2018,13,2018-04-16,16,4,2018,complet certif authent,24k gold plate rose,upper_pop_brand
2,Bundled items requested for Ruie,3,Women/Other/Other,Other,59.0,0,"Banana republic bottoms, Candies skirt with ma...",2018-08-11 00:00:00,6,2018-08-11,11,8,2018,banana republ bottom candi skirt match blazera...,bundl item request ruie,intermediary_brand
3,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,29-1-2018,6,2018-01-29,29,1,2018,size small strap slightli shorten fit x besid ...,acacia pacif tide santorini top,acacia swimwear
4,Black Skater dress,2,"Women/Dresses/Above Knee, Mini",rue,16.0,0,"Xl, great condition",29-11-2018,9,2018-11-29,29,11,2018,xl great condit,black skater dress,rue


In [10]:
filtro_women['sub_category_name'] = sub_categoria(filtro_women)

In [11]:
filtro_women.sub_category_name.value_counts()

Athletic Apparel       52807
Shoes                  46187
Tops & Blouses         39696
Women's Handbags       31017
Jewelry                29635
Women's Accessories    27216
Dresses                24673
Sweaters               22042
Jeans                  19653
Underwear              15462
Coats & Jackets        11447
Swimwear               10567
Skirts                  7728
Pants                   6178
Other                   4685
Maternity               2586
Suits & Blazers         1427
Name: sub_category_name, dtype: int64

# Modelo 01
Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name, shipping, item_condition_id e brands_filter

In [12]:
import statsmodels.api as sm

In [13]:
filtro_women = pd.get_dummies(filtro_women, columns=['sub_category_name','brands_filter','item_condition_id','shipping'])

In [14]:
X = filtro_women.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_women.price
X = sm.add_constant(X)
m1_rl = sm.OLS(exog=X,endog=y)
m1_result = m1_rl.fit()
print(m1_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     701.7
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        17:47:01   Log-Likelihood:            -1.7686e+06
No. Observations:              353006   AIC:                         3.538e+06
Df Residuals:                  352675   BIC:                         3.542e+06
Df Model:                         330                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [15]:
filtro_women['price'].describe()

count    353006.000000
mean         33.838341
std          46.696731
min           3.000000
25%          14.000000
50%          22.000000
75%          39.000000
max        2006.000000
Name: price, dtype: float64

In [18]:
m1_ypred = m1_result.predict(X)
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

36.28

# Modelo 02

Regressão Linear com restrição de preço ate 500 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, shipping, item-condition_id e brands_filter

In [19]:
filtro_women_ajustado = filtro_women[filtro_women['price'] <= 500]

In [21]:
X1 = filtro_women_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_women_ajustado.price
X1 = sm.add_constant(X1)
m2_rl = sm.OLS(exog=X1,endog=y1)
m2_result = m2_rl.fit()
print(m2_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     813.1
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        17:49:06   Log-Likelihood:            -1.6636e+06
No. Observations:              352584   AIC:                         3.328e+06
Df Residuals:                  352253   BIC:                         3.331e+06
Df Model:                         330                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [22]:
filtro_women_ajustado['price'].describe()

count    352584.000000
mean         32.887526
std          35.962911
min           3.000000
25%          13.000000
50%          22.000000
75%          39.000000
max         500.000000
Name: price, dtype: float64

In [23]:
m2_ypred = m2_result.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

27.09

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [24]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [25]:
X2 = filtro_women.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_women.price
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")

R² = 0.49
MSE = 1108.57
RMSE = 33.3


# Modelo 04

Arvore de Regressão com restrição de preço ate 500 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, shipping, item_condition_id e brands_filter

In [26]:
X3 = filtro_women_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_women_ajustado.price
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.53
MSE = 605.07
RMSE = 24.6


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 39,6% | 36,28 | Regressão Linear |
| Modelo 02 | 43,2% | 27,09 | Regressão Linear (preço <= 500) |
| Modelo 03 | 49,0% | 33,30 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 53,0% | 24,60 | Arvore Regressão (criterio = R², preço <= 500) |